In [5]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_excel(r"C:\Users\bartd\Erasmus\Erasmus_\Jaar 4\Master Econometrie\Seminar\UnileverCase\Data\UFS_Universe_NL.xlsx")
data.describe()

,Latitude,Longitude
count,29055.000000,29055.000000
mean,52.077413,5.233738
std,0.529786,0.760728
min,50.755089,3.363958
25%,51.748478,4.663847
50%,52.087704,5.090103
75%,52.369291,5.829936
max,53.497267,7.218729


In [10]:
data.columns

Index(['operatorId', 'name', 'address', 'postalCode', 'city', 'Latitude',
       'Longitude', 'globalChannel', 'cuisineType', 'closed'],
      dtype='object')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29055 entries, 0 to 29054
Data columns (total 10 columns):
operatorId       29055 non-null object
name             29055 non-null object
address          29055 non-null object
postalCode       29046 non-null object
city             29040 non-null object
Latitude         29055 non-null float64
Longitude        29055 non-null float64
globalChannel    29055 non-null object
cuisineType      29055 non-null object
closed           24281 non-null object
dtypes: float64(2), object(8)
memory usage: 1.3+ MB


In [14]:
missing_data = pd.DataFrame({'total_missing': data.isnull().sum(), 'perc_missing': (data.isnull().sum()/len(data))*100})
missing_data

NameError: name 'mc_df' is not defined

In [30]:
import pandas as pd
import numpy as np

In [31]:
#READ DATA
data = pd.read_excel(r"C:\Users\bartd\Erasmus\Erasmus_\Jaar 4\Master Econometrie\Seminar\UnileverCase\Data\UFS_Universe_NL.xlsx")
data.head()

,operatorId,name,address,postalCode,city,Latitude,Longitude,globalChannel,cuisineType,closed
0,/ ChIJ--0P4kpLx0cRO2iA0Sf6ZkY /,'t Brunke,"Sint Nicolaasstraat 2, 6041 EH Roermond, Nethe...",6041 EH,Roermond,51.195881,5.984512,Cafe,Unknown,OPERATIONAL
1,2795484429 / ChIJ--17ntTjxUcR7UDCPpIqMWE /,Enfes,Pieter Calandlaan,1068NH,Amsterdam,52.353989,4.810633,Fast food restaurant,Unknown,OPERATIONAL
2,/ ChIJ--3jn-IkyEcRh88iTuMs7BA /,de Bakker & de Schenker,"Brink 15, 9401 HT Assen, Netherlands",9401 HT,Assen,52.994075,6.562568,Other Restaurant,Unknown,OPERATIONAL
3,3000334749 / ChIJ--KqKRR5xkcRQBF8Ghfl6Qg /,Theetuin De Kwakel,Noord-Linschoterzandweg,3425EL,Snelrewaard,52.029636,4.881653,Cafe,Unknown,OPERATIONAL
4,/ ChIJ--aBe7kJxkcRj3waPQO288U / eb7015b6-a137...,Restaurant Looks,"Binnen Bantammerstraat 7, 1011 CH Amsterdam, N...",1011 CH,Amsterdam,52.373925,4.902064,Other Restaurant,Unknown,CLOSED_PERMANENTLY


In [32]:
#See different columns
data.columns

Index(['operatorId', 'name', 'address', 'postalCode', 'city', 'Latitude',
       'Longitude', 'globalChannel', 'cuisineType', 'closed'],
      dtype='object')

In [33]:
#Check Datatypes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29055 entries, 0 to 29054
Data columns (total 10 columns):
operatorId       29055 non-null object
name             29055 non-null object
address          29055 non-null object
postalCode       29046 non-null object
city             29040 non-null object
Latitude         29055 non-null float64
Longitude        29055 non-null float64
globalChannel    29055 non-null object
cuisineType      29055 non-null object
closed           24281 non-null object
dtypes: float64(2), object(8)
memory usage: 1.3+ MB


In [34]:
#Sure that we won't need this...
data.set_index('name', drop = True, inplace= True)
data.drop(columns = ['operatorId'], inplace = True)

In [35]:
#Closed is categorical variable
data.closed = data.closed.astype('category')

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29055 entries, 't Brunke to Steinbusch / L H E
Data columns (total 8 columns):
address          29055 non-null object
postalCode       29046 non-null object
city             29040 non-null object
Latitude         29055 non-null float64
Longitude        29055 non-null float64
globalChannel    29055 non-null object
cuisineType      29055 non-null object
closed           24281 non-null category
dtypes: category(1), float64(2), object(5)
memory usage: 1.1+ MB


In [37]:
#What kind of missing values are there?
for column in ['globalChannel', 'cuisineType', 'closed']:
    print(data.loc[:, column].unique())
    
#For globalChannel we might say that other restaurant is unknown
#For cuisineType Unkown is the missing value
#For closed nan is the missing value
# data = data.replace(['Unknown', 'Other Restaurant' ], np.nan)

['Cafe' 'Fast food restaurant' 'Other Restaurant' 'Wine Bar' 'Bistro'
 'Retail Bakery' 'Strandtent' 'Bar' 'Pizzeria' 'Pannenkoekenhuis'
 'Tea Shops' 'Burger restaurant' 'Food delivery restaurant' 'Snackbar'
 'Steakhouse']
['Unknown' 'african' 'italian' 'greek' 'indonesian' 'chinese' 'mexican'
 'indian' 'japanese' 'shoarma; italian' 'shoarma' 'thai'
 'vietnamese; japanese' 'afghan' 'korean; thai' 'kebab'
 'japanese; chinese' 'turkish' 'moroccan' 'vietnamese; thai; chinese'
 'peruvian' 'nepal; greek' 'spanish' 'vietnamese; chinese'
 'italian; mexican' 'thai; chinese' 'vietnamese' 'nepal'
 'korean; japanese' 'shoarma; kebab' 'german' 'korean' 'indian; kebab'
 'turkish; kebab' 'turkish; mexican' 'indian; afghan' 'korean; chinese'
 'turkish; italian' 'turkish; italian; kebab' 'shoarma; spanish'
 'indonesian; chinese' 'italian; kebab' 'italian; greek' 'thai; italian'
 'kebab; mexican']
[OPERATIONAL, CLOSED_PERMANENTLY, NaN, CLOSED_TEMPORARILY]
Categories (3, object): [OPERATIONAL, CLOSED_PER

In [41]:
#How much missing data is there?
missing_obs = data.isin([np.nan,'Unknown', 'Other Restaurant']).sum()
missing_data = pd.DataFrame({'total_missing':missing_obs, 'perc_missing': (missing_obs/len(data))*100})
missing_data

,total_missing,perc_missing
address,0,0.000000
postalCode,0,0.000000
city,0,0.000000
Latitude,0,0.000000
Longitude,0,0.000000
globalChannel,16157,66.580129
cuisineType,22758,93.781679
closed,0,0.000000


In [39]:
#Observations without dependent variable are copmletely useless
data = data[data['closed'].notna()]

In [40]:
#Shows restaurants that don't have postal code nor City adress
print(data.postalCode[data.postalCode.isna()])
print(data.city[data.city.isna()])
data = data[data.postalCode.notna()]
data = data[data.city.notna()]

name
AVRAM                 NaN
VOF Het Hapje         NaN
Sultanim Döner        NaN
CAFE H. HEUTINCK      NaN
Metro Döner           NaN
CHINA - GARDEN        NaN
Restaurant Robuust    NaN
Cafe De Wolthoorn     NaN
Name: postalCode, dtype: object
name
Columbus automatisering                  NaN
nationaal militair museum                NaN
Café de Pub                              NaN
Restaurant Feestzaal In den Wijngaard    NaN
GaiaZoo Kerkrade                         NaN
Friterie Par Hasard                      NaN
Name: city, dtype: object


In [42]:
#Recategorize global channel and onehotencode it
print(data.loc[:, 'globalChannel'].value_counts())

Other Restaurant            16157
Cafe                         4314
Retail Bakery                1552
Bar                           978
Fast food restaurant          624
Pizzeria                      325
Burger restaurant             173
Food delivery restaurant      130
Bistro                         11
Pannenkoekenhuis                3
Name: globalChannel, dtype: int64


In [43]:
#Make categories
no_dining = ['Retail Bakery', 'Cafe', 'Bar', 'Wine Bar']
fastfood = ['Food delivery restaurant', 'Fast food restaurant']
other = ['Other Restaurant', 'Bistro', 'Pannenkoekenhuis']

In [44]:
#Create new variable with new categories
data.loc[data.globalChannel.isin(no_dining),'globalChannelShort'] = 'no_dining'
data.loc[data.globalChannel.isin(fastfood),'globalChannelShort'] ='fastfood'
data.loc[data.globalChannel.isin(other),'globalChannelShort'] = 'other'

In [45]:
#One hot encode
data.globalChannelShort = data.globalChannelShort.astype('category')
data = pd.get_dummies(data, columns = ['globalChannelShort'], prefix = ['globalChannel'])

data.head()

,address,postalCode,city,Latitude,Longitude,globalChannel,cuisineType,closed,globalChannel_fastfood,globalChannel_no_dining,globalChannel_other
name,,,,,,,,,,,
't Brunke,"Sint Nicolaasstraat 2, 6041 EH Roermond, Nethe...",6041 EH,Roermond,51.195881,5.984512,Cafe,Unknown,OPERATIONAL,0,1,0
Enfes,Pieter Calandlaan,1068NH,Amsterdam,52.353989,4.810633,Fast food restaurant,Unknown,OPERATIONAL,1,0,0
de Bakker & de Schenker,"Brink 15, 9401 HT Assen, Netherlands",9401 HT,Assen,52.994075,6.562568,Other Restaurant,Unknown,OPERATIONAL,0,0,1
Theetuin De Kwakel,Noord-Linschoterzandweg,3425EL,Snelrewaard,52.029636,4.881653,Cafe,Unknown,OPERATIONAL,0,1,0
Restaurant Looks,"Binnen Bantammerstraat 7, 1011 CH Amsterdam, N...",1011 CH,Amsterdam,52.373925,4.902064,Other Restaurant,Unknown,CLOSED_PERMANENTLY,0,0,1


In [47]:
data['pc4'] = data.postalCode.str.extract('(\d+)').astype(int) #obtain postcode column 

#Check if every postcode is four elements
# [len(zipcode) for zipcode in data.pc4]
for index, zipcode in data.pc4.items():
    try:
        if len(str(zipcode))!= 4:
            print(zipcode)
            print(data.loc[index, :])
    except:
        print(zipcode)

0
address                    0000 ZZ Rotterdam, Netherlands
postalCode                                        0000 ZZ
city                                            Rotterdam
Latitude                                          51.8821
Longitude                                         4.54989
globalChannel                            Other Restaurant
cuisineType                                       Unknown
closed                                 CLOSED_PERMANENTLY
globalChannel_fastfood                                  0
globalChannel_no_dining                                 0
globalChannel_other                                     1
pc4                                                     0
Name: Dushi \'Bon Dia\' Caribische eetgelegenheid, dtype: object
